In [ ]:
import nltk 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, gutenberg
porter = PorterStemmer()

In [ ]:
nltk.download('stopwords')
stopWords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
BLOCKSIZE=100000

In [ ]:
import re
def symbolremoval(word):
  return re.sub('[^A-Za-z0-9\s]+', '', word).lower()

In [ ]:
from collections import defaultdict
def bsbi():
  termdoc_list = defaultdict(list)
  with open('gutenberg_data.csv') as csv_file:
    next(csv_file)
    csv_files = csv.reader(csv_file)
    fileno = 0
    i = 0
    current_block = 0
    for row in csv_files:
      #creating the term-DocID relation
      title, author, link, id, bookshelf, body = row
      i+=1
      content = body.split()
      for word in content:
        word = symbolremoval(word)
        if word and word not in stopWords:
          word = porter.stem(word)
          if word not in termdoc_list:
            current_block += 1
          
          if id not in termdoc_list[word]:
            termdoc_list[word].append(id)
            current_block += 1
          #sorting the term-DocID relation and creating its posting lists
          if current_block >= BLOCKSIZE:
            word_sorted = sorted(termdoc_list.items(), key=lambda x:x[0])
            #creating block files
            with open(f'./IRLAB/Lab2BLOCK{fileno}.txt', 'w') as myfile:
              for id, docs in word_sorted:
                myfile.write(id)
                for docid in docs:
                  myfile.write(f' {docid}')
                myfile.write('\n')
            
            current_block = 0
            termdoc_list.clear()
            fileno += 1
    print("blocks computed")    

    wordsorted = sorted(termdoc_list.items(), key=lambda x:x[0])
    if len(wordsorted)>0:
      with open(f'./IRLAB/Lab2BLOCK{fileno}.txt', 'w') as myfile:
        for id, docs in wordsorted:
          myfile.write(id)
          for docid in docs:
            myfile.write(f' {docid}')
          myfile.write('\n')
      current_block = 0
      termdoc_list.clear()
      fileno += 1     
    print(i)

In [ ]:
bsbi()


15 files done
23 files done
41 files done
54 files done
67 files done
81 files done
96 files done
116 files done
130 files done
146 files done
157 files done
181 files done
207 files done
220 files done
254 files done
271 files done
280 files done
291 files done
302 files done
332 files done
353 files done
371 files done
387 files done
400 files done
444 files done
456 files done
468 files done
476 files done
488 files done
499 files done
506 files done
509 files done
518 files done
526 files done
533 files done


KeyboardInterrupt: ignored

In [ ]:
#find the block in the disk for adding into priority queue
%cd 
file = [f'./IRLAB/Lab2BLOCK{i}.txt' for i in range(947)]
fileseek = [open(i) for i in file]
drive.mount('/content/drive')
#merging the different block files created
#into one merged file using heap queue
import heapq

#creating a generator such so as to split the words and their doc_ids 
#in the posting list of each word
def decorated_file(f, key):
  for word in f:
    yield (key(word), word)
  
files = map(open, file)
outfile = open('./IRLAB/Lab2merged.txt', 'w')

#spliting the words
def key_fn(word):
  return word.split(' ',2)[0]

wordpresent = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
  #checking if the word already exists or not
  if wordpresent != line[0]:
    #if not add the new word to the file
    outfile.write(f'\n{line[1].strip()}')
    wordpresent = line[0]
  else:
    #else append the new data into the word's list
    outfile.write(f'{line[1][len(line[0]):].strip()}')
for i in fileseek:
  i.close()
outfile.close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored